In [1]:
import tensorflow as tf
import numpy as np
import tf_utils

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
np.random.seed(1)

In [3]:
X_train,y_train,X_test,y_test,classes = tf_utils.load_dataset()

In [4]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [5]:
m, height, width, channels = X_train.shape

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

In [7]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [8]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [9]:
def max_pool_2d(x,ksize, strides):
    return tf.nn.max_pool(x, ksize=ksize, strides=strides, padding='SAME')

In [10]:
x = tf.placeholder(tf.float32, [None, 64, 64, 3], name='x')

In [11]:
y = tf.placeholder(tf.float32, [None, 6], name='x')

In [12]:
y_train = convert_to_one_hot(y_train, 6)
y_test = convert_to_one_hot(y_test, 6)
X_train = X_train / 255
X_test = X_test / 255

In [13]:
W1 = weight_variable([5, 5, 3, 8])
b1 = bias_variable([8])

In [14]:
h_conv1 = tf.nn.relu(conv2d(x, W1) + b1)
h_pool1 = max_pool_2d(h_conv1,[1,2,2,1],[1,2,2,1])

In [15]:
W2 = weight_variable([5,5,8,16])
b2 = bias_variable([16])

In [16]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W2) + b2)
h_pool2 = max_pool_2d(h_conv2,[1,2,2,1],[1,2,2,1])

In [17]:
W_fc1 = weight_variable([16*16*16, 1024])
b_fc1 = bias_variable([1024])

In [18]:
h_flat = tf.reshape(h_pool2, [-1, 16*16*16])
h_fc1 = tf.nn.relu(tf.matmul(h_flat, W_fc1)+ b_fc1)

In [19]:
keep_prob = tf.placeholder(tf.float32)

In [20]:
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [24]:
W_fc2 = weight_variable([1024, 72])
b_fc2 = bias_variable([72])

In [25]:
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2)+b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

In [26]:
W_fc3 = weight_variable([72, 6])
b_fc3 = bias_variable([6])

In [27]:
y_conv = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3)+b_fc3, name='output')

In [41]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [42]:
def next_batch(train_data, train_target, batch_size):  
    index = [ i for i in range(0,len(train_target)) ]  
    np.random.shuffle(index);  
    batch_data = []; 
    batch_target = [];  
    for i in range(0,batch_size):  
        batch_data.append(train_data[index[i]]);  
        batch_target.append(train_target[index[i]])  
    return batch_data, batch_target  

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch_x, batch_y = next_batch(X_train, y_train.T, 64)
        _,c = sess.run([optimizer, cost], feed_dict={x:batch_x, y:batch_y, keep_prob:0.5})
        if (i%100 == 0 or i==1999) and i!=0:
            acc = sess.run(accuracy,feed_dict={x:batch_x, y:batch_y, keep_prob:1.0})
            print('迭代%d 正确率为%f' %(i+1, acc))
            acc = sess.run(accuracy,feed_dict={x:X_test, y:y_test.T, keep_prob:1.0})
            print('测试集准确率:%f  损失率%f'%(acc, c))
    constant_graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, ['output'])
    with tf.gfile.FastGFile('./model/model.pb', mode='wb') as f:
        f.write(constant_graph.SerializeToString())

迭代101 正确率为0.406250
测试集准确率:0.333333  损失率1.750180
迭代201 正确率为0.515625
测试集准确率:0.550000  损失率1.625495
迭代301 正确率为0.718750
测试集准确率:0.725000  损失率1.500522
迭代401 正确率为0.875000
测试集准确率:0.791667  损失率1.352206


In [50]:
  with tf.Graph().as_default():
        x2 = tf.placeholder(tf.float32, [None, 64, 64, 3], name='input')
        y2 = tf.placeholder(tf.float32, [None, 6])
        W1_2 = tf.constant(_W1, name='const_W1')
        b1_2 = tf.constant(_b1, name='const_b1')
        h_conv1_2 = tf.nn.relu(conv2d(x2, W1_2) + b1_2)
        h_pool1_2 = max_pool_2d(h_conv1_2,[1,2,2,1],[1,2,2,1])
        
        W2_2 = tf.constant(_W2, name='const_W2')
        b2_2 = tf.constant(_b2, name='const_b2')
        h_conv2_2 = tf.nn.relu(conv2d(h_pool1_2, W2_2) + b2_2)
        h_pool2_2 = max_pool_2d(h_conv2_2,[1,2,2,1],[1,2,2,1])
        
        W_fc1_2 = tf.constant(_W_fc1, name='const_W_fc1')
        b_fc1_2 = tf.constant(_b_fc1, name='const_b_fc1')
        h_flat_2 = tf.reshape(h_pool2_2, [-1,16*16*16])
        h_fc1_2 = tf.nn.relu(tf.matmul(h_flat_2, W_fc1_2)+b_fc1_2)
        
        keep_prob_2 = tf.placeholder(tf.float32)
        h_drop1_2 = tf.nn.dropout(h_fc1_2, keep_prob_2)
        
        W_fc2_2 = tf.constant(_W_fc2, name='const_W_fc2')
        b_fc2_2 = tf.constant(_b_fc2, name='const_b_fc2')
        h_fc2_2 = tf.nn.relu(tf.matmul(h_drop1_2, W_fc2_2)+b_fc2_2)
        h_drop2_2 = tf.nn.dropout(h_fc2_2, keep_prob_2)
        
        W_fc3_2 = tf.constant(_W_fc3, name='const_W_fc3')
        b_fc3_2 = tf.constant(_b_fc3, name='const_b_fc3')
        y_conv_2 = tf.nn.softmax(tf.matmul(h_drop2_2, W_fc3_2)+b_fc3_2, name='output')
        with tf.Session() as sess_2:
            sess_2.run(tf.global_variables_initializer())
            tf.train.write_graph(sess_2.graph_def, './model/', 'model.pb', as_text=False)
            #correct_prediction_2 = tf.equal(tf.argmax(y_conv_2, 1), tf.argmax(y_2, 1))
            #accuracy_2 = tf.reduce_mean(tf.cast(correct_prediction_2, tf.float32))